In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
from tqdm import tqdm
import math

# Fix random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [7]:

# 1. Enhanced Data Loading and Preprocessing
def load_data():
    train_df = pd.read_csv('train_smote.csv')
    test_df = pd.read_csv('test_smote.csv')

    # Feature scaling (normalization)
    train_features = train_df.drop(columns=['failure mode']).values.astype(np.float32)
    test_features = test_df.drop(columns=['failure mode']).values.astype(np.float32)

    # Normalize using training set statistics
    feature_mean = train_features.mean(axis=0)
    feature_std = train_features.std(axis=0) + 1e-8  # Avoid division by zero
    train_features = (train_features - feature_mean) / feature_std
    test_features = (test_features - feature_mean) / feature_std

    # Labels (assuming classes start from 1)
    train_labels = (train_df['failure mode'].values - 1).astype(np.int64)
    test_labels = (test_df['failure mode'].values - 1).astype(np.int64)

    # ===== 新增数据验证 =====
    print("\n=== 数据形状验证 ===")
    print(f"训练数据形状: {train_features.shape}")
    print(f"训练标签形状: {train_labels.shape}")
    print(f"测试数据形状: {test_features.shape}")
    print(f"测试标签形状: {test_labels.shape}")
    print(f"唯一标签值: {np.unique(train_labels)}")

    assert train_features.shape[1] == test_features.shape[1], "训练和测试特征维度不一致！"
    assert len(np.unique(train_labels)) == len(np.unique(test_labels)), "训练和测试标签类别不一致！"


    return (train_features, train_labels), (test_features, test_labels)

(train_features, train_labels), (test_features, test_labels) = load_data()

# 2. Improved Bayesian Neural Network Implementation
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_sigma1=1.0, prior_sigma2=0.1, prior_pi=0.5):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Weight parameters (mean and rho)
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).normal_(0, 0.1))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-5, -4))

        # Bias parameters (mean and rho)
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).normal_(0, 0.1))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5, -4))

        # Scale mixture prior parameters
        self.prior_sigma1 = prior_sigma1
        self.prior_sigma2 = prior_sigma2
        self.prior_pi = prior_pi

        # Initialize prior distributions
        self.weight_prior = self.scale_mixture_prior()
        self.bias_prior = self.scale_mixture_prior()

        # For KL divergence approximation
        self.kl = 0
    '''
    def scale_mixture_prior(self):
        return torch.distributions.Normal(0.0, torch.tensor([self.prior_sigma1, self.prior_sigma2]))
    '''
    def scale_mixture_prior(self):
    # 创建尺度混合先验分布
        mix = torch.distributions.Categorical(torch.tensor([self.prior_pi, 1.0 - self.prior_pi]))
        comp = torch.distributions.Normal(
            torch.tensor([0.0, 0.0]),
            torch.tensor([self.prior_sigma1, self.prior_sigma2])
        )
        return torch.distributions.MixtureSameFamily(mix, comp)
    '''
    def forward(self, x, sample=True):
        if sample:
            # Sample weights and biases using reparameterization trick
            weight_sigma = torch.log1p(torch.exp(self.weight_rho))
            bias_sigma = torch.log1p(torch.exp(self.bias_rho))

            eps_weight = torch.randn_like(self.weight_mu)
            eps_bias = torch.randn_like(self.bias_mu)

            weight = self.weight_mu + weight_sigma * eps_weight
            bias = self.bias_mu + bias_sigma * eps_bias

            # Compute KL divergence
            self.kl = self.kl_divergence(weight, weight_sigma, bias, bias_sigma)
        else:
            weight = self.weight_mu
            bias = self.bias_mu

        return F.linear(x, weight, bias)
    '''
    def forward(self, x, sample=True):
    # 输入维度验证
        assert x.size(-1) == self.in_features, \
        f"输入特征维度{x.size(-1)}与层输入特征数{self.in_features}不匹配"

        if sample:
            weight_sigma = torch.log1p(torch.exp(self.weight_rho))
            bias_sigma = torch.log1p(torch.exp(self.bias_rho))

        # 采样时添加形状验证
            assert self.weight_mu.shape == (self.out_features, self.in_features), \
            f"权重mu形状应为{(self.out_features, self.in_features)}，实际为{self.weight_mu.shape}"
            assert self.bias_mu.shape == (self.out_features,), \
            f"偏置mu形状应为{(self.out_features,)}, 实际为{self.bias_mu.shape}"

            eps_weight = torch.randn_like(self.weight_mu)
            eps_bias = torch.randn_like(self.bias_mu)

            weight = self.weight_mu + weight_sigma * eps_weight
            bias = self.bias_mu + bias_sigma * eps_bias

            self.kl = self.kl_divergence(weight, weight_sigma, bias, bias_sigma)
        else:
            weight = self.weight_mu
            bias = self.bias_mu

        output = F.linear(x, weight, bias)

    # 输出维度验证
        assert output.size(-1) == self.out_features, \
        f"输出特征维度{output.size(-1)}与层输出特征数{self.out_features}不匹配"

        return output

    def kl_divergence(self, weight, weight_sigma, bias, bias_sigma):
        '''
        # Posterior distributions
        weight_post = torch.distributions.Normal(self.weight_mu, weight_sigma)
        bias_post = torch.distributions.Normal(self.bias_mu, bias_sigma)

        # KL divergence for weights
        kl_weight = torch.distributions.kl.kl_divergence(weight_post, self.weight_prior).sum()
        kl_bias = torch.distributions.kl.kl_divergence(bias_post, self.bias_prior).sum()
        '''
        # 权重后验分布
        weight_post = torch.distributions.Normal(self.weight_mu, weight_sigma)
    # 偏置后验分布
        bias_post = torch.distributions.Normal(self.bias_mu, bias_sigma)

    # 计算权重KL散度
        kl_weight = torch.sum(torch.distributions.kl.kl_divergence(
            weight_post,
            torch.distributions.Normal(0, self.prior_sigma1)
        ))

    # 计算偏置KL散度
        kl_bias = torch.sum(torch.distributions.kl.kl_divergence(
            bias_post,
            torch.distributions.Normal(0, self.prior_sigma1)
        ))
        return kl_weight + kl_bias


=== 数据形状验证 ===
训练数据形状: (676, 69)
训练标签形状: (676,)
测试数据形状: (43, 69)
测试标签形状: (43,)
唯一标签值: [0 1 2 3]


In [3]:
class BayesianMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], output_dim=4,
                 dropout_prob=0.1, prior_sigma1=1.0, prior_sigma2=0.1, prior_pi=0.5):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Create Bayesian layers
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(BayesianLinear(prev_dim, hidden_dim, prior_sigma1, prior_sigma2, prior_pi))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))
            prev_dim = hidden_dim

        self.layers = nn.Sequential(*layers)
        self.output_layer = BayesianLinear(prev_dim, output_dim, prior_sigma1, prior_sigma2, prior_pi)

        # Total KL divergence
        self.kl = 0

         # ===== 新增模型结构验证 =====
        print("\n=== 模型结构验证 ===")
        print(f"输入维度: {input_dim}")
        print(f"隐藏层维度: {hidden_dims}")
        print(f"输出维度: {output_dim}")

        # 测试前向传播
        test_input = torch.randn(2, input_dim)  # 2个样本
        print(f"\n测试输入形状: {test_input.shape}")
        test_output = self.forward(test_input)
        print(f"测试输出形状: {test_output.shape}")
        assert test_output.shape == (2, output_dim), f"模型输出形状应为(2, {output_dim})，实际得到{test_output.shape}"

    def forward(self, x, sample=True):
        self.kl = 0
        x = self.layers(x)
        x = self.output_layer(x, sample)
        self.kl += self.output_layer.kl
        return x

    def get_kl(self):
        return self.kl

In [4]:

# 3. Enhanced Training and Evaluation Functions
def train_epoch(model, dataloader, optimizer, criterion, scaling_factor, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(dataloader, desc="Training", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # Forward pass with sampling
        outputs = model(inputs, sample=True)
        loss = criterion(outputs, targets)

        # Add KL divergence
        kl = model.get_kl()
        total_loss = loss + scaling_factor * kl

        # Backward pass and optimize
        total_loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss.item() / len(dataloader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

def evaluate_model(model, dataloader, criterion, scaling_factor, device, num_samples=100):
    model.eval()
    all_probs = []
    all_targets = []
    total_loss = 0.0

    # Get all targets first
    with torch.no_grad():
        for _, targets in dataloader:
            all_targets.append(targets.cpu().numpy())
    all_targets = np.concatenate(all_targets)

    # Monte Carlo sampling
    for _ in tqdm(range(num_samples), desc="Evaluating", leave=False):
        sample_probs = []
        sample_loss = 0.0

        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs, sample=True)
            loss = criterion(outputs, targets)
            kl = model.get_kl()
            sample_loss += (loss + scaling_factor * kl).item()

            probs = F.softmax(outputs, dim=1)
            sample_probs.append(probs.detach().cpu().numpy())

        all_probs.append(np.concatenate(sample_probs))
        total_loss += sample_loss / len(dataloader)

    # Calculate metrics
    mean_probs = np.mean(all_probs, axis=0)
    preds = np.argmax(mean_probs, axis=1)

    avg_loss = total_loss / num_samples
    acc = accuracy_score(all_targets, preds)
    prec = precision_score(all_targets, preds, average='macro', zero_division=0)
    rec = recall_score(all_targets, preds, average='macro', zero_division=0)
    f1 = f1_score(all_targets, preds, average='macro', zero_division=0)

    # Calculate uncertainty metrics
    entropy = -np.sum(mean_probs * np.log(mean_probs + 1e-10), axis=1)
    avg_entropy = np.mean(entropy)

    return {
        'loss': avg_loss,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'entropy': avg_entropy,
        'probs': mean_probs
    }

In [5]:

# 4. Hyperparameter Optimization with Bayesian Approach
class HyperparameterOptimizer:
    def __init__(self, param_space, num_iter=20):
        self.param_space = param_space
        self.num_iter = num_iter
        self.results = []

    def suggest_params(self):
        params = {}
        for key, values in self.param_space.items():
            if isinstance(values[0], (int, float)):
                # Continuous parameter - sample from log space if specified
                if 'log' in key:
                    low = math.log10(values[0])
                    high = math.log10(values[1])
                    val = 10 ** random.uniform(low, high)
                else:
                    val = random.uniform(values[0], values[1])
            else:
                # Discrete parameter
                val = random.choice(values)
            params[key] = val
        return params

    def optimize(self, train_features, train_labels, device, n_folds=5):
        best_score = -np.inf
        best_params = None

        for i in range(self.num_iter):
            params = self.suggest_params()
            print(f"\nIteration {i+1}/{self.num_iter}: Testing params {params}")

            # Cross-validation
            kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
            cv_scores = []

            for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(train_labels)))):
                print(f"  Fold {fold+1}/{n_folds}")

                # Create data loaders
                train_dataset = TensorDataset(
                    torch.tensor(train_features[train_idx]),
                    torch.tensor(train_labels[train_idx])
                )
                val_dataset = TensorDataset(
                    torch.tensor(train_features[val_idx]),
                    torch.tensor(train_labels[val_idx])
                )

                train_loader = DataLoader(train_dataset, batch_size=int(params['batch_size']), shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=int(params['batch_size']), shuffle=False)

                # Initialize model
                hidden_dims = [int(params['hidden_dim1']), int(params['hidden_dim2']), int(params['hidden_dim3'])]
                model = BayesianMLP(
                    input_dim=train_features.shape[1],
                    hidden_dims=hidden_dims,
                    output_dim=len(np.unique(train_labels)),
                    dropout_prob=params['dropout'],
                    prior_sigma1=params['prior_sigma1'],
                    prior_sigma2=params['prior_sigma2'],
                    prior_pi=params['prior_pi']
                ).to(device)

                optimizer = optim.Adam(model.parameters(), lr=params['lr'])
                criterion = nn.CrossEntropyLoss()

                # Train for a few epochs
                for epoch in range(int(params['epochs_per_fold'])):
                    train_loss, train_acc = train_epoch(
                        model, train_loader, optimizer, criterion,
                        params['scaling_factor'], device
                    )
                    print(f"    Epoch {epoch+1}: Train Loss {train_loss:.4f}, Acc {train_acc:.2f}%")

                # Evaluate
                val_results = evaluate_model(
                    model, val_loader, criterion,
                    params['scaling_factor'], device, num_samples=20
                )
                cv_scores.append(val_results['f1'])
                print(f"    Val F1: {val_results['f1']:.4f}")

            mean_score = np.mean(cv_scores)
            self.results.append((params, mean_score))

            if mean_score > best_score:
                best_score = mean_score
                best_params = params
                print(f"New best params found with F1: {best_score:.4f}")

        return best_params, best_score

In [6]:

# 5. Main Training Pipeline
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Define hyperparameter search space
    param_space = {
        'lr': [1e-4, 1e-3],  # Learning rate
        'batch_size': [16, 32, 64],
        'hidden_dim1': [128, 256, 512],
        'hidden_dim2': [64, 128, 256],
        'hidden_dim3': [32, 64, 128],
        'dropout': [0.1, 0.2, 0.3],
        'scaling_factor': [1e-5, 1e-4, 1e-3],  # KL scaling
        'prior_sigma1': [0.5, 1.0, 2.0],
        'prior_sigma2': [0.01, 0.1, 0.5],
        'prior_pi': [0.25, 0.5, 0.75],
        'epochs_per_fold': [5, 10]
    }

    # Hyperparameter optimization
    optimizer = HyperparameterOptimizer(param_space, num_iter=10)
    best_params, best_score = optimizer.optimize(train_features, train_labels, device)

    print("\nBest Hyperparameters:")
    print(best_params)
    print(f"Best Validation F1: {best_score:.4f}")

    # Train final model with best params
    print("\nTraining final model...")
    train_dataset = TensorDataset(torch.tensor(train_features), torch.tensor(train_labels))
    train_loader = DataLoader(train_dataset, batch_size=int(best_params['batch_size']), shuffle=True)

    test_dataset = TensorDataset(torch.tensor(test_features), torch.tensor(test_labels))
    test_loader = DataLoader(test_dataset, batch_size=int(best_params['batch_size']), shuffle=False)

    final_model = BayesianMLP(
        input_dim=train_features.shape[1],
        hidden_dims=[int(best_params['hidden_dim1']),
                     int(best_params['hidden_dim2']),
                     int(best_params['hidden_dim3'])],
        output_dim=len(np.unique(train_labels)),
        dropout_prob=best_params['dropout'],
        prior_sigma1=best_params['prior_sigma1'],
        prior_sigma2=best_params['prior_sigma2'],
        prior_pi=best_params['prior_pi']
    ).to(device)

    final_optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'])
    criterion = nn.CrossEntropyLoss()

    # Training loop
    best_val_f1 = 0
    for epoch in range(50):  # Train for more epochs
        train_loss, train_acc = train_epoch(
            final_model, train_loader, final_optimizer,
            criterion, best_params['scaling_factor'], device
        )

        # Evaluate on test set periodically
        if (epoch + 1) % 5 == 0:
            test_results = evaluate_model(
                final_model, test_loader, criterion,
                best_params['scaling_factor'], device, num_samples=50
            )

            print(f"\nEpoch {epoch+1}:")
            print(f"  Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
            print(f"  Test Results:")
            print(f"    Loss: {test_results['loss']:.4f}")
            print(f"    Accuracy: {test_results['accuracy']:.4f}")
            print(f"    Precision: {test_results['precision']:.4f}")
            print(f"    Recall: {test_results['recall']:.4f}")
            print(f"    F1: {test_results['f1']:.4f}")
            print(f"    Avg Entropy: {test_results['entropy']:.4f}")

    # Final evaluation with more samples
    print("\nFinal Evaluation with 100 MC samples:")
    final_results = evaluate_model(
        final_model, test_loader, criterion,
        best_params['scaling_factor'], device, num_samples=100
    )

    print("\n--- Final Test Results ---")
    print(f"Accuracy: {final_results['accuracy']:.4f}")
    print(f"Precision: {final_results['precision']:.4f}")
    print(f"Recall: {final_results['recall']:.4f}")
    print(f"F1-Score: {final_results['f1']:.4f}")
    print(f"Average Predictive Entropy: {final_results['entropy']:.4f}")

if __name__ == "__main__":
    main()

Using device: cpu

Iteration 1/10: Testing params {'lr': 0.0006754841186120954, 'batch_size': 16.400172083562673, 'hidden_dim1': 163.20375275124726, 'hidden_dim2': 78.28548724152466, 'hidden_dim3': 55.5670788532484, 'dropout': 0.16766994874229113, 'scaling_factor': 9.02961610934361e-05, 'prior_sigma1': 0.543469416314708, 'prior_sigma2': 0.04797296377167434, 'prior_pi': 0.25744930485951756, 'epochs_per_fold': 6.093189874018017}
  Fold 1/5

=== 模型结构验证 ===
输入维度: 69
隐藏层维度: [163, 78, 55]
输出维度: 4

测试输入形状: torch.Size([2, 69])
测试输出形状: torch.Size([2, 4])


    Epoch 1: Train Loss 0.0331, Acc 47.04%


    Epoch 2: Train Loss 0.0230, Acc 72.04%


    Epoch 3: Train Loss 0.0162, Acc 80.93%


    Epoch 4: Train Loss 0.0132, Acc 85.74%


...


Epoch 35:
  Train Loss: 0.0011, Acc: 99.41%
  Test Results:
    Loss: 0.5273
    Accuracy: 0.9070
    Precision: 0.8986
    Recall: 0.8070
    F1: 0.8059
    Avg Entropy: 0.0949



Epoch 40:
  Train Loss: 0.0013, Acc: 99.26%
  Test Results:
    Loss: 0.5130
    Accuracy: 0.9070
    Precision: 0.8986
    Recall: 0.8070
    F1: 0.8059
    Avg Entropy: 0.0760



Epoch 45:
  Train Loss: 0.0010, Acc: 99.26%
  Test Results:
    Loss: 0.6980
    Accuracy: 0.8605
    Precision: 0.7333
    Recall: 0.7792
    F1: 0.7395
    Avg Entropy: 0.0769



Epoch 50:
  Train Loss: 0.0009, Acc: 99.70%
  Test Results:
    Loss: 0.4852
    Accuracy: 0.9070
    Precision: 0.8986
    Recall: 0.8070
    F1: 0.8059
    Avg Entropy: 0.0721

Final Evaluation with 100 MC samples:



--- Final Test Results ---
Accuracy: 0.9070
Precision: 0.8986
Recall: 0.8070
F1-Score: 0.8059
Average Predictive Entropy: 0.0724
